<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">TP : La cuisine du système</h1>

On souhaite observer le fonctionnement de l'ordonnanceur du systéme d'exploitation en prenant la métaphore d'une cuisine de restaurant.

On considère des programmes Python qui seront des recettes de cuisine.  
Nous utiliserons les fichiers d'un répertoire `ingredients` pour gérer notre garde-manger, des fichiers d'un repertoire `ustensiles` pour gérer les ressources de la cuisine (four, batteur, etc) et enfin les fichiers d'un répertoire `plats` pour compter les plats produits par les cuisiniers.  
Nous nous interesserons aux accès concurrents qui peuvet être faits sur ces ressources, aux problèmes qui peuvent en découler et à l'utilisation des verrous pour éviter ces problèmes.  
Enfin, avec les verrous, nous illustrerons une situation d'interblocage.

## Ordonnancement et concurrence
Dans un premier temps, nous allons réaliser une bibliothèque [`actions.py`](Fichiers/actions.py) qui implémente une fonction pour chacun des gestes de cuisine que les recettes peuvent demander de réaliser.

Par exemple, le fichier [`actions.py`](Fichiers/actions.py) définit la fonction :

In [ ]:
def prendre_un_abricot(pourquoi):
    print("Prendre un abricot,", pourquoi)
    return prendre("ingredients/abricot")

Cette fonction affiche un message indiquant qu'un abricot est utilisé et en précise l'usage, puis appelle `prendre()` qui décrémente le nombre d'abricots écrits dans le fichier `abricots` du repertoire `ingredients`.

La fonction `prendre()` ouvre le fichier passé en paramètre, lit le nombre stocké dans ce fichier, décrémente ce nombre, puis écrit cette valeur du nombre d'ingrédients disponibles dans le même fichier :

In [ ]:
def prendre(filename):
    file = open(filename)
    quantite = int(file.readline())
    file.close()
    if quantite > 0: 
       quantite = quantite - 1
    file = open(filename,"w")
    file.write(str(quantite))
    file.close()
    return quantite

Nous allons réaliser une première recette qui consiste simplement, pour le cuisinier, à manger un abricot.

Cette recette sera implémentée dans un fichier [`manger_un_abricot.py`](Fichiers/manger_un_abricot.py) comme suit :

In [ ]:
import actions

abricots = actions.prendre_un_abricot("pour le manger !")
print("Il reste", abricots)

Pour que cette recette puisse être réalisée, il faut, préalablement, créer le répertoire `ingredients`, puis mettre des abricots dans ce *garde-manger*.

    utilisateur@machine:~$ mkdir ingredients
    utilisateur@machine:~$ echo 100 > ingredients/abricot

L'éxecution de la recette produit la trace suivante : 

    utilisateur@machine:~$ python3 manger_un_abricot.py
    Prendre un abricot, pour le manger !
    Il reste 99

Il est possible de faire faire cette recette à deux cuisiniers, simultanément, avec la (double) commande suivante :

    utilisateur@machine:~$ python3 manger_un_abricot.py & python3 manger_un_abricot.py & 

Si tout se passe bien, cela produit la trace suivante : 

    [1] 2617469
    [2] 2617470
    utilisateur@machine:~$ Prendre un abricot, pour le manger !
    Il reste 98
    Prendre un abricot, pour le manger !
    Il reste 97

    [1]-  Fini                    python3 manger_un_abricot.py
    [2]+  Fini                    python3 manger_un_abricot.py

Cette trace montre bien que deux programmes se sont exécutés en même temps.  
Chacun a décrémenté le compteur du garde-manger et il ne reste que 97 abricots.

Pourtant, en re-exécutant plusieurs fois cette double commande, il est possible que l'on obtienne une trace *anormale* telle que celle-ci :

    utilisateur@machine:~$ python3 manger_un_abricot.py & python3 manger_un_abricot.py & 
    [1] 2710223
    [2] 2710224
    utilisateur@machine:~$ Prendre un abricot, pour le manger !
    Prendre un abricot, pour le manger !
    Il reste 96
    Il reste 96

    [1]-  Fini                    python3 manger_un_abricot.py
    [2]+  Fini                    python3 manger_un_abricot.py

Il semble alors que le programme ne fonctionne pas correctement.  
En effet, deux exécutions se sont déroulées et elles ont toutes deux décrémentées le nombre d'abricots, et pourtant, un seul abricot a été *mangé*.

Cette situation n'est pas systématique : elle se produit parfois, et, parfois, elle ne se produit pas...  
On dit que ce comportement n'est pas *déterministe* (ou qu'il est *indéterministe*).

1 . Pour comprendre ce qui s'est produit, considérer la fonction `prendre()`, qui a été utilisée par les deux programmes.  
Proposer un ordonnancement des deux programmes qui aurait pu faire passer le nombre d'abricots de 97 à 95, puis un autre qui aurait pu faire passer le nombre d'abricots de 97 à 96. 

2 . Sachant que les deux situations sont possibles, expliquer pourquoi le comportement observé n'est pas déterministe.

Il apparaît donc que le fait de lancer deux programmes en même temps compromet leur bon fonctionnement.  
Pour éviter ce problème, il pourrait sembler légitime de s'interdire de lancer deux programmes en même temps.

3 . En filant la métaphore de la cuisine, expliquer pourquoi le restaurant n'a pas intérêt à réaliser les recettes les unes après les autres s'il y a deux cuisiniers, et même s'il n'y en a qu'un.

Cet exemple montre que la fonction `prendre()` doit limiter l'accès au fichier à un seul programme à la fois.

4 . En utilisant les verrous de la bibliothèque [`filelock`](https://py-filelock.readthedocs.io/en/latest/index.html), proposer une modification de la fonction `prendre()` afin que, quels que soient les choix de l'ordonnancement du système d'exploitation, le nombre d'abricots restant soit toujours correct.

5 . Enrichir maintenant le nombre d'actions possibles afin qu'il soit possible de prendre du beurre, de la farine, du sucre et des oeufs.

## Concurrence et interblocage
Pour pouvoir exécuter une recette, en plus des ingrédients, il faut disposer d'ustensiles.  
Dans ce TP, il s'agit d'un four et d'un batteur.  
Pour pouvoir utiliser un ustensile, il faut le choisir, puis l'utiliser et, une fois la préparation terminée, le libérer.

Ainsi dans le fichier [recette_pate_brisee.py](Fichiers/recette_pate_brisee.py), la recette de la pâte brisée s'écrit :

In [ ]:
recette = "pour faire la pâte brisée"

actions.prendre_du_beurre(recette)
actions.prendre_de_la_farine(recette)
actions.prendre_du_sucre(recette)

batteur = actions.choisir_le_batteur()
actions.utiliser(batteur, recette)

print("mélanger les ingrédients", recette)
sleep(2.0)

four = actions.choisir_le_four()
actions.utiliser(four, recette)

print("cuisson en cours", recette)
sleep(5.0)
print("fin de cuisson", recette)

actions.liberer(four, recette)
actions.liberer(batteur, recette)

Dans cette recette, le cuisinier utilise le four pendant 5 unités de temps, pour faire cuire son fond de tarte.  
En plus des actions, la recette utilise la fonction [`sleep()`](https://docs.python.org/fr/3/library/time.html#time.sleep) de la bibliotèque [`time`](https://docs.python.org/fr/3/library/time.html) pour attendre que la cuisson soit terminée.  
Les autres actions de notre bibliothèque [`actions`](Fichiers/actions.py) peuvent être définies comme suit : 

In [ ]:
def choisir_le_four():
    return "four", FileLock("ustensiles/four.lock")

def choisir_le_batteur():
    return "batteur", FileLock("ustensiles/batteur.lock")

def utiliser(ustensile, pourquoi):
    ustensile[1].acquire()
    print("Utiliser le", ustensile[0], pourquoi)

def liberer(ustensile, pourquoi):
    print("Nettoyer le", ustensile[0], pourquoi)
    ustensile[1].release()

Les ustensiles sont rangés dans le répertoire `ustensiles` qu'il faut préalablement créer.  
Et pour que l'hygiène d'une cuisine professionnelle soit irréprochable, on nettoie les ustensiles dès que l'on n'en a plus l'usage.

Pour la suite, nous considérons aussi la recette de la crème patissière : 

In [ ]:
recette = "pour faire la crème pâtissière"

actions.prendre_du_sucre(recette)
four = actions.choisir_le_four()
actions.utiliser(four, recette)

print("caramélisation en cours", recette)
sleep(2.0)
print("caramel prêt", recette)

actions.prendre_du_lait(recette)
actions.prendre_de_la_farine(recette)
actions.prendre_un_oeuf(recette)

batteur = actions.choisir_le_batteur()
actions.utiliser(batteur, recette)

print("mélanger les ingrédients", recette)
sleep(2.0)
print("ajouter les ingrédients au caramel dans le four", recette)
sleep(2.0)
print("cuisson terminée", recette)

actions.liberer(four, recette)
actions.liberer(batteur, recette)

1 . Vérifier d'abord que lorsque l'on réalise une recette, puis la suivante, tout se passe bien.  

2 . Que se passe-t-il quand on réalise simultanément les deux recettes?  
Expliquer ce que l'on observe.

Pour mémoire, pour forcer la terminaision d'un programme, on peut utiliser la commande [`kill`](https://pubs.opengroup.org/onlinepubs/9699919799/utilities/kill.html) en précisant le numéro du ṕrocessu à *tuer*.

3 . Expliquer ce que l'on observe lorsque l'on exécute la commande `kill` sur l'un des deux programmes.

4 . Proposer une modification de l'une des deux recettes pour que cette situation ne se reproduise plus.

## Ressources partagées et files d'attente
Pour l'instant, nous avons considéré que le four ne pouvait être utilisé que par une personne à la fois.  
Cependant les fours professionels sont suffisament grands pour accueillir plusieurs plats en même temps.

Nous allons maintenant considérer le cas dans lequel le four pourrait accueillir deux plats avant d'être plein.

Pour cela, nous nous proposons de modifier le comportement de `choisir_xxx()` et `utiliser()` :
* lorsque l'on utilise un ustensile, on décrémente le nombre de places disponibles ; s'il ne reste plus de place, on prend le verrou
* lorsqu'on le libère, on incrémente le nombre de places disponibles et, s'il reste moins d'une place, on libère le verrou

Ainsi un cuisinier attendra un ustensile seulement s'il n'ay a plus de place.  
Nous mettrons le compteur dans un fichier `ustensile`.

Par contre, comme le compteur du nombre de place est incrémenté ou décrémenté potentiellement par plusieurs programmes en même temps, ces opérations doivent être gardées par un verrou (différent du premier).

1 . Implémenter `ustensile`pour permettre d'utiliser un four disposant de 2 places et de 2 batteurs.  

2 . Vérifier l'implémentation en lançant deux recettes en même temps, puis trois.

Vous venez d'implémenter un **sémaphore**.  
Les sémaphores sont des verrous enrichis d'un compteur qui définit au bout de combien d'utilisations simultanées la ressource devient indisponible.  
Les sémaphores sont des mécanismes de synchronisation proposés par les systèmes d'exploitation, mais ils peuvent être mis en oeuvre avec une utilisation appropriée de deux verrous.